<a href="https://colab.research.google.com/github/shenben/metrics/blob/master/crawl_short_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!sudo apt update
!sudo apt install ./google-chrome-stable_current_amd64.deb
!wget https://storage.googleapis.com/chrome-for-testing-public/129.0.6668.100/linux64/chromedriver-linux64.zip
# https://googlechromelabs.github.io/chrome-for-testing/
!unzip chromedriver-linux64.zip
!sudo mv chromedriver-linux64/chromedriver /usr/local/bin/
!sudo chmod +x /usr/local/bin/chromedriver
!chromedriver --version


--2024-10-12 09:08:07--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 74.125.26.190, 74.125.26.136, 74.125.26.93, ...
Connecting to dl.google.com (dl.google.com)|74.125.26.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 110925276 (106M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 105.79M   162MB/s    in 0.7s    

2024-10-12 09:08:08 (162 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [110925276/110925276]

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:6 https://r

In [7]:
!pip install selenium pytube3 opencv-python youtube-dl yt_dlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.3/171.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 51.6 MB/s eta 0:00:00


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Set Chrome options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Set path to chromedriver if necessary
chrome_service = Service('/usr/local/bin/chromedriver')

# Initialize WebDriver
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# Test WebDriver
driver.get("https://www.google.com")
print(driver.title)
driver.quit()


Google


In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import yt_dlp
import time
import os

# Configure WebDriver (make sure you have ChromeDriver installed)
def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    # Set path to chromedriver if necessary
    chrome_service = Service('/usr/local/bin/chromedriver')

    # Initialize WebDriver
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
    return driver

# Function to search for videos
def search_videos(driver, query):
    # driver.get('https://www.youtube.com/results?search_query=' + query)
    driver.get('https://www.youtube.com/results?search_query=' + query + '&sp=EgQQARgB')

    time.sleep(2)
    video_elements = driver.find_elements(By.XPATH, '//a[@id="video-title"]')
    video_links = [video.get_attribute('href') for video in video_elements if video.get_attribute('href')]
    return video_links[:50]  # Return first 10 video links

# # Function to download YouTube videos using yt-dlp
# def download_video(url, output_path):
#     try:
#         ydl_opts = {
#             'format': 'best',
#             'cookies-from-browser':'chrome',
#             'outtmpl': output_path + '/%(title)s.%(ext)s',  # Save with video title
#         }
#         with yt_dlp.YoutubeDL(ydl_opts) as ydl:
#             ydl.download([url])
#         return True
#     except Exception as e:
#         print(f"Failed to download video {url}: {e}")
#         return False

def download_video(urls, output_path):
    try:
        ydl_opts = {
            'format': 'best[ext=mp4]',  # Limit to MP4 format for better compatibility
            'cookies-from-browser': 'chrome',
            'outtmpl': output_path + '/%(title)s.%(ext)s',
            'max-filesize': 50 * 1024 * 1024,  # Max file size 50MB
            'postprocessors': [{
                'key': 'FFmpegVideoConvertor',
                'preferedformat': 'mp4',  # Convert to mp4 if needed
            }],
            'match-filters': 'duration < 240',  # Only download videos under 4 minutes
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download(urls)
        return True
    except Exception as e:
        print(f"Failed to download video {urls}: {e}")
        return False


if __name__ == "__main__":
    # Setup Chrome WebDriver
    driver = setup_driver()

    # Search for videos based on specific keywords
    queries = ['outdoor nature', 'sports activities', 'product unboxing']
    all_video_links = []
    for query in queries:
        all_video_links.extend(search_videos(driver, query))
        # break
    with open('short_video_urls.txt', 'w') as f:
      for line in all_video_links:
          f.write(f"{line}\n")

    # Download and process videos
    output_dir = "./downloaded_videos"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # for video_url in all_video_links:
    #     if download_video(video_url, output_dir):
    #         print(f"Video downloaded successfully: {video_url}")
    #     else:
    #         print(f"Skipping unsuitable video {video_url}")
    #     break

    driver.quit()


In [9]:
with open('short_video_urls.txt', 'r') as f:
    urls = [line.strip() for line in f if line.strip()]

download_video(urls, "./downloaded_videos")
# # Download videos in batch
# with yt_dlp.YoutubeDL(ydl_opts) as ydl:
#     ydl.download(urls)

[youtube] Extracting URL: https://www.youtube.com/shorts/PchDOqEG6xo
[youtube] PchDOqEG6xo: Downloading webpage


[youtube] PchDOqEG6xo: Downloading ios player API JSON


[youtube] PchDOqEG6xo: Downloading iframe API JS


[youtube] PchDOqEG6xo: Downloading mweb player API JSON


ERROR: [youtube] PchDOqEG6xo: Failed to extract any player response; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


Failed to download video ['https://www.youtube.com/shorts/PchDOqEG6xo', 'https://www.youtube.com/shorts/evQcasJrCg8', 'https://www.youtube.com/shorts/77g9cz_X-7Q', 'https://www.youtube.com/shorts/kEzYF7Ac7bw', 'https://www.youtube.com/watch?v=A2CwmzcGXnI&pp=ygUOb3V0ZG9vciBuYXR1cmU%3D', 'https://www.youtube.com/shorts/Z8mJDNcLjYc', 'https://www.youtube.com/shorts/zcBz5oVNSt0', 'https://www.youtube.com/shorts/mSQVwrmc2Vc', 'https://www.youtube.com/watch?v=hmFQqjMF_f0&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D', 'https://www.youtube.com/watch?v=JUpxmE1MuQQ&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D', 'https://www.youtube.com/watch?v=XPo-UkZCDMY&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D', 'https://www.youtube.com/watch?v=ZXnHE1tZQjo&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D', 'https://www.youtube.com/watch?v=ejwItt1rNek&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D', 'https://www.youtube.com/watch?v=u_J5SSybBxE&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D', 'https://www.youtube.com/shorts/katrwpAF4Kk', 'https://www.youtube.com/watch?v=WCYTlVF-dj

False

In [ ]:
yt-dlp -f 'bestvideo+bestaudio/best' https://www.youtube.com/watch?v=ipf7ifVSeDU


In [ ]:
!yt-dlp -f 'bestvideo+bestaudio/best' https://www.youtube.com/watch?v=W86cTIoMv2U

In [ ]:
!touch video_urls.txt

In [27]:
import yt_dlp

# Options for downloading best video + audio
ydl_opts = {
    'format': 'bestvideo+bestaudio/best',
    'outtmpl': '%(title)s.%(ext)s',
    'cookies': '/content/cookies.txt'
}
# ydl_opts = {
#     'format': 'bestvideo+bestaudio/best',
#     'outtmpl': 'downloaded_videos/%(title)s.%(ext)s',  # Save with video title
#     'merge_output_format': 'mp4',
#     'cookies': '/content/cookies.txt'
# }

# Read URLs from the text file
# with open('video_urls.txt', 'r') as f:
with open('short_video_urls.txt', 'r') as f:
    urls = [line.strip() for line in f if 'watch' in line]
    # urls = [video_url.replace('/shorts/', '/watch?v=') for video_url in urls]
    # urls = [for k in urls]

print(urls)
# Download videos in batch
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download(urls)


['https://www.youtube.com/watch?v=hmFQqjMF_f0&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D', 'https://www.youtube.com/watch?v=JUpxmE1MuQQ&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D', 'https://www.youtube.com/watch?v=XPo-UkZCDMY&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D', 'https://www.youtube.com/watch?v=ZXnHE1tZQjo&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D', 'https://www.youtube.com/watch?v=ejwItt1rNek&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D', 'https://www.youtube.com/watch?v=u_J5SSybBxE&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D', 'https://www.youtube.com/watch?v=WCYTlVF-djw&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D', 'https://www.youtube.com/watch?v=mooOEpoQffA&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D', 'https://www.youtube.com/watch?v=sxUj4uMNu8g&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D', 'https://www.youtube.com/watch?v=27f0QUUScQ8&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D', 'https://www.youtube.com/watch?v=9zG8ATPLuTM&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D', 'https://www.youtube.com/watch?v=jh_ACOy_3Fc&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D', 'https://www.youtube.com/wa

[youtube] hmFQqjMF_f0: Downloading ios player API JSON


[youtube] hmFQqjMF_f0: Downloading iframe API JS


[youtube] hmFQqjMF_f0: Downloading mweb player API JSON


ERROR: [youtube] hmFQqjMF_f0: Failed to extract any player response; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


DownloadError: ERROR: [youtube] hmFQqjMF_f0: Failed to extract any player response; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U

In [ ]:
def download_video(url, output_path):
    try:
        ydl_opts = {
            'format': 'best[ext=mp4]',  # Restrict to mp4
            'outtmpl': output_path + '/%(title)s.%(ext)s',  # Save with video title
            'cookies': './cookies.txt',  # Path to cookies file
            'max-filesize': 50 * 1024 * 1024,  # Max 50MB size
            'match-filters': 'duration < 240',  # Only videos shorter than 4 minutes
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        return True
    except Exception as e:
        print(f"Failed to download video {url}: {e}")
        return False


In [12]:
!yt-dlp https://www.youtube.com/shorts/PchDOqEG6xo


[youtube] Extracting URL: https://www.youtube.com/shorts/PchDOqEG6xo
[youtube] PchDOqEG6xo: Downloading webpage
[youtube] PchDOqEG6xo: Downloading ios player API JSON
[youtube] PchDOqEG6xo: Downloading mweb player API JSON
[youtube] PchDOqEG6xo: Downloading player 2f238d39
[youtube] PchDOqEG6xo: Downloading m3u8 information
[info] PchDOqEG6xo: Downloading 1 format(s): 303+251
[download] Destination: Enjoying outdoors 🌳 #tennessee #outdoor #nature [PchDOqEG6xo].f303.webm
[download] 100% of    4.19MiB in 00:00:01 at 3.51MiB/s
[download] Destination: Enjoying outdoors 🌳 #tennessee #outdoor #nature [PchDOqEG6xo].f251.webm
[download] 100% of  245.47KiB in 00:00:00 at 710.88KiB/s
[Merger] Merging formats into "Enjoying outdoors 🌳 #tennessee #outdoor #nature [PchDOqEG6xo].webm"
Deleting original file Enjoying outdoors 🌳 #tennessee #outdoor #nature [PchDOqEG6xo].f251.webm (pass -k to keep)
Deleting original file Enjoying outdoors 🌳 #tennessee #outdoor #nature [PchDOqEG6xo].f303.webm (pass -k t

In [24]:
!yt-dlp --cookies cookies.txt https://www.youtube.com/watch?v=hmFQqjMF_f0&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D


[youtube] Extracting URL: https://www.youtube.com/watch?v=hmFQqjMF_f0
[youtube] hmFQqjMF_f0: Downloading webpage
[youtube] hmFQqjMF_f0: Downloading ios player API JSON
[youtube] hmFQqjMF_f0: Downloading mweb player API JSON
[youtube] hmFQqjMF_f0: Downloading m3u8 information
[info] hmFQqjMF_f0: Downloading 1 format(s): 616+251
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 44
[download] Destination: How playing sports benefits your body ... and your brain - Leah Lagos and Jaspal Ricky Singh [hmFQqjMF_f0].f616.mp4
[download] 100% of   37.30MiB in 00:00:05 at 6.61MiB/s
[download] Destination: How playing sports benefits your body ... and your brain - Leah Lagos and Jaspal Ricky Singh [hmFQqjMF_f0].f251.webm
[download] 100% of    4.00MiB in 00:00:00 at 20.38MiB/s
[Merger] Merging formats into "How playing sports benefits your body ... and your brain - Leah Lagos and Jaspal Ricky Singh [hmFQqjMF_f0].webm"
Deleting original file How playing sports benefits your body ... 

In [30]:
import yt_dlp

ydl_opts = {
    'format': 'bestvideo+bestaudio/best',
    'outtmpl': '%(title)s.%(ext)s',
    'cookies': './cookies.txt',
    'verbose': True,
    'dump_single_json': True  # Dumps all information about the video
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=hmFQqjMF_f0'])



[debug] Encodings: locale UTF-8, fs utf-8, pref UTF-8, out UTF-8 (No ANSI), error UTF-8 (No ANSI), screen UTF-8 (No ANSI)
[debug] yt-dlp version stable@2024.10.07 from yt-dlp/yt-dlp [1a176d874] (pip) API
[debug] params: {'format': 'bestvideo+bestaudio/best', 'outtmpl': '%(title)s.%(ext)s', 'cookies': './cookies.txt', 'verbose': True, 'dump_single_json': True, 'compat_opts': set(), 'http_headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.106 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'Accept-Language': 'en-us,en;q=0.5', 'Sec-Fetch-Mode': 'navigate'}}
[debug] Python 3.10.12 (CPython x86_64 64bit) - Linux-6.1.85+-x86_64-with-glibc2.35 (OpenSSL 3.0.2 15 Mar 2022, glibc 2.35)
[debug] exe versions: ffmpeg 4.4.2 (setts), ffprobe 4.4.2
[debug] Optional libraries: Cryptodome-3.21.0, brotli-1.1.0, certifi-2024.08.30, mutagen-1.47.0, requests-2.32.3, secretstorage-3.3.1, sqli

[youtube] Extracting URL: https://www.youtube.com/watch?v=hmFQqjMF_f0
[youtube] hmFQqjMF_f0: Downloading webpage


[youtube] hmFQqjMF_f0: Downloading ios player API JSON


[youtube] hmFQqjMF_f0: Downloading iframe API JS


[youtube] hmFQqjMF_f0: Downloading mweb player API JSON


ERROR: [youtube] hmFQqjMF_f0: Failed to extract any player response; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U
  File "/usr/local/lib/python3.10/dist-packages/yt_dlp/extractor/common.py", line 741, in extract
    ie_result = self._real_extract(url)
  File "/usr/local/lib/python3.10/dist-packages/yt_dlp/extractor/youtube.py", line 4365, in _real_extract
    webpage, master_ytcfg, player_responses, player_url = self._download_player_responses(url, smuggled_data, video_id, webpage_url)
  File "/usr/local/lib/python3.10/dist-packages/yt_dlp/extractor/youtube.py", line 4329, in _download_player_responses
    player_responses, player_url = self._extract_player_responses(
  File "/usr/local/lib/python3.10/dist-packages/yt_dlp/extractor/youtube.py", line 3988, in _extract_player_responses
    raise ExtractorError('Failed to extract any player response')



DownloadError: ERROR: [youtube] hmFQqjMF_f0: Failed to extract any player response; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U

In [31]:
import subprocess

def download_video(url):
    command = [
        'yt-dlp', '--cookies', './cookies.txt', '--format', 'bestvideo+bestaudio/best', url
    ]
    try:
        subprocess.run(command, check=True)
        print(f"Downloaded: {url}")
    except subprocess.CalledProcessError as e:
        print(f"Failed to download {url}: {e}")

# Batch download
with open('short_video_urls.txt', 'r') as f:
    urls = [line.strip() for line in f]

for url in urls:
    try:
      download_video(url)
    except:
      print("error:",url)

Downloaded: https://www.youtube.com/watch?v=hmFQqjMF_f0&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D
Downloaded: https://www.youtube.com/watch?v=JUpxmE1MuQQ&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D
Downloaded: https://www.youtube.com/watch?v=XPo-UkZCDMY&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D
Downloaded: https://www.youtube.com/watch?v=ZXnHE1tZQjo&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D
Downloaded: https://www.youtube.com/watch?v=ejwItt1rNek&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D
Downloaded: https://www.youtube.com/watch?v=u_J5SSybBxE&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D
Downloaded: https://www.youtube.com/shorts/katrwpAF4Kk
Downloaded: https://www.youtube.com/watch?v=WCYTlVF-djw&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D
Downloaded: https://www.youtube.com/watch?v=mooOEpoQffA&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D
Downloaded: https://www.youtube.com/watch?v=sxUj4uMNu8g&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D
Downloaded: https://www.youtube.com/watch?v=27f0QUUScQ8&pp=ygURc3BvcnRzIGFjdGl2aXRpZXM%3D
Downloaded: https://www.youtube.com/watch?v=9

In [33]:
import os
for i in os.listdir('.'):
  if i.endswith("webm"):
    print(i)

Packing my mini bag with mini products! #asmr #packingessentials #ulta #packingwithme [1djcQeAAhro].webm
UBGoregaon Sports Day 2014-15 Sr.Kg. A - Obstacle Race [ejwItt1rNek].webm
Apple Studio Display + M2 Mac mini & accessories! Unboxing ASMR [Faa2szY337M].webm
Amazing Sports activities on Cruise Ship ｜ Don't Miss ｜ Cordelia Cruises [sxUj4uMNu8g].webm
Sports Song   Educational Children Song   Learning English Sports for Kids [27f0QUUScQ8].webm
MacBook Pro 13” Unboxing 🤍 #unboxing #apple #macbook #asmr #aesthetic [1Ni_5PG4S9w].webm
Unboxing Cololight B-roll 4K ｜ Sony A7III [vL1k-qb4RbE].webm
Sports and Activities - الرياضات والنشاطات ｜ we can 5 وي كان [XPo-UkZCDMY].webm
Enjoying outdoors 🌳 #tennessee #outdoor #nature [PchDOqEG6xo].webm
🇭🇹NEW  NATIONAL RECORD🇭🇹- 19.17m (62’9”) - 16lbs Shotput [katrwpAF4Kk].webm
Games Ideas for Students ｜｜ games for School sports day ｜｜ The wisdom school [jh_ACOy_3Fc].webm
unboxing the first samples of my product! 👀 #shorts [z5xDl67_Kmc].webm
BRATZ Mini M

In [34]:
!sudo apt update && sudo apt install ffmpeg
!pip install moviepy

Get:1 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:8 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,090 B]
Ign:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 2,915 B in 2s (1,939 B/s)
Reading package lists... Done
Building de

In [35]:
import os
import subprocess
from moviepy.editor import VideoFileClip
import pandas as pd

# Directory containing downloaded videos
VIDEO_DIR = '.'

# Convert all .webm videos to .mp4
def convert_webm_to_mp4():
    for filename in os.listdir(VIDEO_DIR):
        if filename.endswith('.webm'):
            input_path = os.path.join(VIDEO_DIR, filename)
            output_path = os.path.join(VIDEO_DIR, filename.replace('.webm', '.mp4'))

            # Run ffmpeg command to convert video
            command = ['ffmpeg', '-i', input_path, '-c:v', 'copy', '-c:a', 'aac', output_path]
            subprocess.run(command, check=True)
            print(f"Converted {filename} to {output_path}")

# Analyze video duration and resolution
def analyze_videos():
    video_data = []

    for filename in os.listdir(VIDEO_DIR):
        if filename.endswith('.mp4'):
            video_path = os.path.join(VIDEO_DIR, filename)

            # Get video duration and resolution
            try:
                clip = VideoFileClip(video_path)
                duration = clip.duration  # Duration in seconds
                resolution = f"{clip.size[0]}x{clip.size[1]}"  # Width x Height
                clip.close()

                video_data.append({
                    'Filename': filename,
                    'Duration (s)': round(duration, 2),
                    'Resolution': resolution
                })

            except Exception as e:
                print(f"Error processing {filename}: {e}")

    # Create a DataFrame and save the analysis as a CSV report
    df = pd.DataFrame(video_data)
    report_path = 'video_analysis_report.csv'
    df.to_csv(report_path, index=False)
    print(f"Analysis report saved to {report_path}")

if __name__ == '__main__':
    # Convert videos and analyze them
    convert_webm_to_mp4()
    analyze_videos()


Converted Packing my mini bag with mini products! #asmr #packingessentials #ulta #packingwithme [1djcQeAAhro].webm to ./Packing my mini bag with mini products! #asmr #packingessentials #ulta #packingwithme [1djcQeAAhro].mp4
Converted UBGoregaon Sports Day 2014-15 Sr.Kg. A - Obstacle Race [ejwItt1rNek].webm to ./UBGoregaon Sports Day 2014-15 Sr.Kg. A - Obstacle Race [ejwItt1rNek].mp4
Converted Apple Studio Display + M2 Mac mini & accessories! Unboxing ASMR [Faa2szY337M].webm to ./Apple Studio Display + M2 Mac mini & accessories! Unboxing ASMR [Faa2szY337M].mp4
Converted Amazing Sports activities on Cruise Ship ｜ Don't Miss ｜ Cordelia Cruises [sxUj4uMNu8g].webm to ./Amazing Sports activities on Cruise Ship ｜ Don't Miss ｜ Cordelia Cruises [sxUj4uMNu8g].mp4
Converted Sports Song   Educational Children Song   Learning English Sports for Kids [27f0QUUScQ8].webm to ./Sports Song   Educational Children Song   Learning English Sports for Kids [27f0QUUScQ8].mp4
Converted MacBook Pro 13” Unboxing

In [37]:
!zip dataset_sv.zip *.mp4

  adding: Amazing Sports activities on Cruise Ship ｜ Don't Miss ｜ Cordelia Cruises [sxUj4uMNu8g].mp4 (deflated 0%)
  adding: Apple Studio Display + M2 Mac mini & accessories! Unboxing ASMR [Faa2szY337M].mp4 (deflated 0%)
  adding: asmr ⧸ Stationery pal unboxing [jLniS38GFy8].mp4 (deflated 0%)
  adding: ASMR ｜ Unboxing First Order From Alibaba [Y-0r3B0_U4A].mp4 (deflated 0%)
  adding: BRATZ Mini Makeup Unboxing Satisfying Video ASMR! #shorts #asmr [V-wRuNZEfWs].mp4 (deflated 0%)
  adding: Enjoying outdoors 🌳 #tennessee #outdoor #nature [PchDOqEG6xo].mp4 (deflated 0%)
  adding: Excellent Pe games and activities for physical education teacher and school [u_J5SSybBxE].mp4 (deflated 0%)
  adding: Games Ideas for Students ｜｜ games for School sports day ｜｜ The wisdom school [jh_ACOy_3Fc].mp4 (deflated 3%)
  adding: How playing sports benefits your body ... and your brain - Leah Lagos and Jaspal Ricky Singh [hmFQqjMF_f0].mp4 (deflated 0%)
  adding: I Bought a PLAYSTATION 5 From Temu For $20… #